<a href="https://colab.research.google.com/github/nurfnick/Data_Viz/blob/main/RecreatingMasters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recreating the Masters


![Master?](https://preview.redd.it/ldafaqqimpe61.png?auto=webp&s=543ee3bc7509b04c164db6a7acccbc30509ab5b9)

To be able to do this I am going to need sports data.  I think I can find that at pro-hockey reference and [images for the teams](https://www.sportslogos.net/teams/list_by_league/1/National_Hockey_League/NHL/logos/)